# Sorting the Dataset based on Timestamp

In [0]:
import pandas as pd
import numpy as np
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import matplotlib.pyplot as plt

In [6]:
from google.colab import files
uploaded = files.upload()

Saving movies.csv to movies (1).csv
Saving ratings.csv to ratings.csv


In [0]:
rat = pd.read_csv('ratings.csv')
mov = pd.read_csv('movies.csv')

In [0]:
df_combined = pd.merge(rat, mov, on = 'movieId')

In [49]:
rat.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


In [50]:
ts = rat['timestamp'].quantile(0.98)
ts

1529334436.3000002

In [51]:
train_ratings = pd.DataFrame(columns=['userId','movieId','rating'])
validation_ratings = pd.DataFrame(columns=['userId','movieId','rating'])
for i in range(len(rat)):
  if rat['timestamp'].iloc[i] <= ts:
    train_ratings = train_ratings.append([{'userId':rat['userId'].iloc[i],'movieId':rat['movieId'].iloc[i],'rating':rat['rating'].iloc[i]}])
    validation_ratings = validation_ratings.append([{'userId':rat['userId'].iloc[i],'movieId':rat['movieId'].iloc[i],'rating':rat['rating'].iloc[i]}])
  else:
    validation_ratings = validation_ratings.append([{'userId':rat['userId'].iloc[i],'movieId':rat['movieId'].iloc[i],'rating':rat['rating'].iloc[i]}])
    
  if i%10000 == 0:
    print(i, "Completed")

0 Completed
10000 Completed
20000 Completed
30000 Completed
40000 Completed
50000 Completed
60000 Completed
70000 Completed
80000 Completed
90000 Completed
100000 Completed


In [0]:
train_ratings = train_ratings.reset_index(drop=True)
validation_ratings = validation_ratings.reset_index(drop=True)

In [56]:
print(len(train_ratings))
print(len(validation_ratings))

98819
100836


In [0]:
# Remove users in validation set those are not present in Training Set
train_users = train_ratings['userId'].unique()
users_not_in_train_set = []

for i in range(1,611):
  if i in train_users:
    continue
  else:
    users_not_in_train_set.append(i)

In [58]:
users_not_in_train_set

[25, 77, 98, 184, 248, 258, 272, 296, 331, 338, 362, 514, 556, 586, 596]

In [59]:
for i in users_not_in_train_set:
  validation_ratings = validation_ratings[validation_ratings['userId']!=i]
validation_ratings.reset_index(drop=True)
validation_ratings['userId'].unique()

array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
       38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54,
       55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
       72, 73, 74, 75, 76, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
       90, 91, 92, 93, 94, 95, 96, 97, 99, 100, 101, 102, 103, 104, 105,
       106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
       119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131,
       132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144,
       145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157,
       158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
       171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183,
       185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197,
       198, 199, 200, 201, 202, 203, 204, 205, 20

In [71]:
print(len(train_ratings['movieId'].unique()))
print(len(validation_ratings['movieId'].unique()))

9559
9608


In [0]:
# Remove Movies that are not in the Train Set
validation_movies = validation_ratings['movieId'].unique()
train_movies = train_ratings['movieId'].unique()
movies_not_in_train_set = []

for i in validation_movies:
  if i in train_movies:
    continue
  else:
    movies_not_in_train_set.append(i)

In [75]:
movies_not_in_train_set

[166015,
 4282,
 46862,
 111732,
 164280,
 188301,
 190183,
 77795,
 119167,
 130444,
 159779,
 166203,
 179511,
 188751,
 78103,
 185585,
 26713,
 49917,
 102742,
 117364,
 143896,
 150696,
 159976,
 173317,
 179135,
 188675,
 188833,
 189381,
 2656,
 27683,
 95473,
 96007,
 103233,
 104141,
 134524,
 137863,
 151763,
 161354,
 172875,
 176935,
 179119,
 182639,
 187541,
 179813,
 183199,
 184721,
 187717,
 188189,
 189713]

In [76]:
for i in movies_not_in_train_set:
  validation_ratings = validation_ratings[validation_ratings['movieId']!=i]
validation_ratings.reset_index(drop=True)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
5,1,70,3.0
6,1,101,5.0
7,1,110,4.0
8,1,151,5.0
9,1,157,5.0


In [78]:
validation_ratings.loc[validation_ratings['movieId']== 96007]

,userId,movieId,rating


In [79]:
print(len(train_ratings))
print(len(validation_ratings))

98819
99007


In [0]:
# Extra length in Validation is a result of available users rating movies after the timestamp

In [81]:
print('Train Users: ', train_ratings['userId'].nunique())
print('Validation Users: ',validation_ratings['userId'].nunique())
print('Train Movies: ',train_ratings['movieId'].nunique())
print('Validation Movies: ',validation_ratings['movieId'].nunique())

Train Users:  595
Validation Users:  595
Train Movies:  9559
Validation Movies:  9559


In [0]:
train_ratings.to_csv("train_ratings.csv")
files.download("train_ratings.csv")

In [0]:
validation_ratings.to_csv("validation_ratings.csv")
files.download("validation_ratings.csv")

# Preparing the Train and Test Dataset

In [8]:
uploaded = files.upload()

Saving train_ratings.csv to train_ratings.csv
Saving validation_ratings.csv to validation_ratings.csv


In [0]:
tr_ratings = pd.read_csv('train_ratings.csv')
val_ratings = pd.read_csv('validation_ratings.csv')

In [85]:
train_dataset = tr_ratings.pivot_table(index = 'userId', columns = 'movieId', values = 'rating')
train_dataset.fillna(0, inplace=True)
train_dataset.head(10)

movieId,1,2,3,4,5,6,7,8,9,10,...,184349,184471,184791,185029,185031,185135,185435,187593,187595,189547
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,4.0,5.0,3.0,5.0,4.0,4.0,3.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [86]:
test_dataset = val_ratings.pivot_table(index='userId', columns='movieId', values='rating')
test_dataset.fillna(0, inplace=True)
test_dataset.head(10)

movieId,1,2,3,4,5,6,7,8,9,10,...,184349,184471,184791,185029,185031,185135,185435,187593,187595,189547
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,4.0,5.0,3.0,5.0,4.0,4.0,3.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
train_dataset.to_csv("train.csv")
files.download("train.csv")

In [0]:
test_dataset.to_csv('test.csv')
files.download('test.csv')

# Train and Test Dataset Class Implementation

In [3]:
from google.colab import files
uploaded = files.upload()

Saving model.py to model.py
Saving test.csv to test.csv
Saving train.csv to train.csv


In [22]:
from google.colab import files
uploaded = files.upload()

Saving loss.py to loss.py


## Train

In [0]:
class TrainDataset(Dataset):
  def __init__(self, train_file, transform=None):
    self.data = pd.read_csv(train_file)
    self.data = self.data.iloc[:,1:]
    self.transform = transform
    
    if transform is not None:
      self.data = self.transform(np.array(self.data))
      
  def __len__(self):
    return len(self.data[0])
  
  def __getitem__(self, ind):
    user_vector = self.data.data[0][ind]
    
    return user_vector
    

In [0]:
transformations = transforms.Compose([transforms.ToTensor()])
train_dat = TrainDataset('train.csv', transformations)

In [4]:
train_dat.__len__()

595

In [5]:
train_dat.__getitem__(5)

tensor([0., 4., 5.,  ..., 0., 0., 0.], dtype=torch.float64)

## Test

In [0]:
class TestDataset(Dataset):
  def __init__(self, test_file, transform=None):
    self.data = pd.read_csv(test_file)
    self.data = self.data.iloc[:,1:]
    self.transform = transform
    
    if transform is not None:
      self.data = self.transform(np.array(self.data))
      
  def __len__(self):
    return len(self.data[0])
  
  def __getitem__(self, ind):
    user_vector = self.data.data[0][ind]
    
    return user_vector

In [7]:
test_dat = TestDataset('test.csv', transformations)
test_dat.__len__()

595

In [8]:
test_dat.__getitem__(5)

tensor([0., 4., 5.,  ..., 0., 0., 0.], dtype=torch.float64)

# DataLoader

In [0]:
train = DataLoader(dataset=train_dat, batch_size = 128, shuffle=True, num_workers = 1)

In [0]:
test = DataLoader(dataset=test_dat, batch_size=128, shuffle=True, num_workers=1)

In [11]:
# CUDA for PyTorch

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
print(device)

cuda:0


In [0]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tqdm
from loss import MSEloss_with_Mask
from model import AutoEncoder

In [0]:
# layer_sizes = [9559, 512, 512, 1024, 512, 512, 9559]
layer_sizes = [9559, 512, 512, 1024]

In [14]:
from google.colab import files
uploaded = files.upload()

In [0]:
model = AutoEncoder(layer_sizes=layer_sizes, nl_type='selu', is_constrained=True, dp_drop_prob=0.0, last_layer_activations=False)

In [0]:
model = model.cuda()

# Training the Model

In [0]:
criterion = MSEloss_with_Mask()
criterion = criterion.cuda()

In [0]:
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay= 1e-6, momentum = 0.9, nesterov = True)

In [19]:
for epoch in range(2):
  train_loss, valid_loss = [], []
  
  # Training Part
  model.train()
  for i, data in enumerate(train, 0):
    # Get the inputs
    inputs = labels = data
    inputs = inputs.cuda()
    labels = labels.cuda()
    
    inputs = inputs.float()
    labels = labels.float()
    
    # zero the parameter gradients
    optimizer.zero_grad()
    
    # forward + backward + optimize
    outputs = model(inputs)
    outputs = outputs.cuda()
    loss = criterion(outputs,labels)
    loss.backward()
    optimizer.step()
    
    train_loss.append(loss.item())
    
  for i, data in enumerate(test, 0):
    inputs = labels = data
    
    outputs = model(inputs)
    loss = MSEloss_with_Mask(outputs, labels)
    valid_loss.append(loss.item())
    
  print ("Epoch:", epoch+1, "Training Loss: ", np.mean(train_loss), "Valid Loss: ", np.mean(valid_loss))

RuntimeError: ignored

NameError: ignored